In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from dask.distributed import Client

client = Client("tcp://127.0.0.1:36075")
client

In [ ]:
import ftfy
import numpy as np
import pandas as pd
import dask.dataframe as dd

# df = dd.read_csv("../../data/raw/viznet/*.csv")

# def eval_with_catch(x):
#     try:
#         return list(eval(x))
#     except Exception:
#         return [""]


# def clean_str(x):
#     try:
#         return ftfy.fix_text(x)
#     except:
#         return ""


# df["data"] = df["data"].apply(func=eval_with_catch, meta=("data", np.array([""])))
# df = df.explode("data")
# df = df.applymap(lambda x: clean_str(x))

In [ ]:
# import csv

# df[["data"]].to_csv("../../data/train/viznet/all.csv", single_file=True, quoting=csv.QUOTE_ALL, index=False)

In [ ]:
df = dd.read_csv("../../../data/train/viznet/all.csv", keep_default_na=False, dtype=str)

In [ ]:
import csv
import nltk

df = df.persist()

def ngrams(x, n):
    if len(x) >= n:
        return ["".join(ngram) for ngram in nltk.ngrams(list(x), n)]
    else:
        return [x]

    
for i in range(2, 5):
    df[f"{i}_gram"] = df["data"].apply(ngrams, meta=(f"{i}_gram", np.array([""])), n=i)
    ngram_df = df.explode(f"{i}_gram")
    value2count = ngram_df[f"{i}_gram"].value_counts()
    value2count.index.name = "data"
    value2count.to_frame("count").to_csv(
        f"../../../data/train/viznet/ngram/{i}_gram_count.csv",
        single_file=True,
        quoting=csv.QUOTE_ALL,
    )
    df = df.drop(f"{i}_gram", axis=1)